In [1]:
from IPython.display import Image
import pandas as pd
from PIL import Image
from glob import glob

# Raw Sequencing Data Processing Pipeline
## Written By: Kathryn Cogert 10/23/18

Intended for MacOSX users with some familiarity with command consoles

Requirements:

- Software & Scripts
 - FastQC (optional)
 - usearch
- Files
    - Raw Sequences in data/ directory of format:
     - Forward Reads: *_R1_001.fastq
     - Reverse Reads: *_R2_001.fastq
    - mapping file for demultiplexing: usearch_mapping.txt
    - mapping file for seperating related samples into unique files: split_map.txt
    - _split_demux_fastq.py (Inlcuded with this repo, this is a script that seperates samples into unqiue files)
    - Ribosomal Database Project (RPD) .udb file for ZOTU assignments (included with this repo)
 

### Step 1. Unzip raw data

Skipping this time, this data was already merged.

In [3]:
! gunzip -k -f data/*.gz 

### Step 2. Examine quality scores of forward and reverse reads

Examining quality of cleaned sequences instead.


This step is not essential to the pipeline but is a valuable check to see if your sequencing data is of good quality.  This requires the FastQC Program.  If you do not have it, download the linux installer at:

https://www.bioinformatics.babraham.ac.uk/projects/download.html#fastqc 

And follow the linux installation directions, being sure to replace "/path/to/" with the path to the location of the unzipped folder.  This will allow you to run fastqc from the command line



In [2]:
! mkdir fastqc_out
! fastqc data/*001.fastq -outdir fastqc_out
! unzip 'fastqc_out/*.zip' -d fastqc_out -o

mkdir: fastqc_out: File exists
Started analysis of Sam1-13_S29_L002_R1_001.fastq
Approx 5% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 10% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 15% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 20% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 25% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 30% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 35% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 40% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 45% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 50% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 55% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 60% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 65% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 70% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 75% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 80% complete for Sam1-13_S29_L002_R1_001.fastq
Approx 85% complete for Sam1-13_S29_L002_R1_001.fastq
Ap

### Read Quality:

For Illumina sequecning it's typical to see read lengths between 30-350 bp long, note if any sequences in particular are flagged as poor quality.  It is also typical for quality to drop off with read length (dunno why this isn't working, so I did it outside the database this time)

In [3]:
! unzip fastqc_out/*.zip -n
! head -10 KC_fastqc/fastqc_data.txt
img_f = "KC_fastqc/Images/per_base_quality.png"
Image(filename=glob.glob(img_f)[0])


Archive:  fastqc_out/Sam1-13_S29_L002_R1_001_fastqc.zip
caution: filename not matched:  fastqc_out/Sam1-13_S29_L002_R2_001_fastqc.zip
caution: filename not matched:  -n
head: KC_fastqc/fastqc_data.txt: No such file or directory


AttributeError: 'function' object has no attribute 'glob'

### Reverse Read Quality:

Skipping, reads already merged.

Same as above, but it is typical for reverse reads to be slightly worse in quality than forward reads.

! head -10 fastqc_out/*_R2_001_fastqc/fastqc_data.txt
img_f = "fastqc_out/*_R2_001_fastqc/Images/per_base_quality.png"
Image(filename=glob.glob(img_f)[0])

### Step 3. Merge forward and reverse reads


This step aligns the forward and reverse reads,  I have used Sam's recommended minimum overlap of 32 bp.  If you want to be very conservative, you can use 45.


In [11]:
! usearch -fastq_mergepairs data/*_R1_001.fastq -reverse data/*_R2_001.fastq -fastqout merged.fastq -fastq_minovlen 32

usearch v11.0.667_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: cogerk@gmail.com


Merging
  Fwd data/Sam1-13_S29_L002_R1_001.fastq
  Rev data/Sam1-13_S29_L002_R2_001.fastq
  Keep read labels

00:00 2.6Mb  FASTQ base 33 for file data/Sam1-13_S29_L002_R1_001.fastq
01:23 69Mb    100.0% 94.9% merged

Totals:
   4813708  Pairs (4.8M)
   4566682  Merged (4.6M, 94.87%)
   3744358  Alignments with zero diffs (77.79%)
    138962  Too many diffs (> 5) (2.89%)
        95  Fwd tails Q <= 2 trimmed (0.00%)
        12  Rev tails Q <= 2 trimmed (0.00%)
       568  Fwd too short (< 64) after tail trimming (0.01%)
        16  Rev too short (< 64) after tail trimming (0.00%)
    106843  No alignment found (2.22%)
       637  Alignment too short (< 32) (0.01%)
      2934  Staggered pairs (0.06%) merged & trimmed
     83.24  Mean alignment length
    417.59  Mean merged length
      0.37  Mean fwd expected error

### Step 4. Demultiplex data

This data contains many different samples. Each sample has a unique barcode, this step looks for those barcodes in order to seperate the different samples from one another.

This requires a mapping file, usearch_mapping.txt of format:

\> SAMPLE.NAME <br>
ATCCGAAA (I just made up this barcode)

In [12]:
! usearch -fastx_demux merged.fastq -barcodes data/mapping.txt -fastqout demux.fastq

usearch v11.0.667_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: cogerk@gmail.com

00:00 1.6Mb   100.0% Reading data/mapping.txt
00:42 35Mb    100.0% Demuxed 1937982 / 4566682 (42.4%)


### Step 5. Trim barcodes and primers

Now that samples are seperated by barcode, we don't need the barcodes anymore. While we're at it, we can get rid of the primers used in the sequencing.

In [1]:
! usearch -fastx_truncate demux.fastq -stripleft 27 -stripright 20 -fastqout trimmed.fastq

usearch v11.0.667_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: cogerk@gmail.com

00:20 35Mb    100.0% Processing, 0 (0.0%) too short


### Step 6. Split related samples into unqiue files
Skipping, everything is related

We can now put all of the related samples in their own files, this allows us to focus in on the experiment that we want to examine.

! python _split_demux_fastq.py -f trimmed.fastq -m split_map.txt 

### Step 7. Concatenate samples from a different sequence for comparison


In [2]:
! cat data/trimmed.combined_old.fastq trimmed.fastq > All_samples.fastq

### Step 7. Filter sequences with errors out

We will now filter out all sequences with more than 1 error.  This algorithm may also do some additional filtration and cleaning as well.

In [3]:
! usearch -fastq_filter All_samples.fastq -fastq_maxee 1.0 -fastaout filtered.fasta

usearch v11.0.667_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: cogerk@gmail.com

00:00 1.9Mb  FASTQ base 33 for file All_samples.fastq
01:52 35Mb    100.0% Filtering, 92.7% passed
   2552601  Reads (2.6M)                    
    186378  Discarded reads with expected errs > 1.00
   2366223  Filtered reads (2.4M, 92.7%)


### Step 8. Relabel individual sequence

Each sequence is given an index number here.

In [4]:
! usearch -fastx_relabel filtered.fasta -fastaout seqs.fa

usearch v11.0.667_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: cogerk@gmail.com

01:33 35Mb    100.0% Processing


### Step 9. Select unique sequences

Remove duplicate sequences for determining OTUs

In [5]:
! usearch -fastx_uniques seqs.fa -fastaout unique_seqs.fa -sizeout -relabel Uniq

usearch v11.0.667_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: cogerk@gmail.com

00:11 407Mb   100.0% Reading seqs.fa
00:21 1.1Gb   100.0% DF
00:22 846Mb  2366223 seqs, 582905 uniques, 487788 singletons (83.7%)
00:22 846Mb  Min size 1, median 1, max 113386, avg 4.06
00:44 1.1Gb   100.0% Writing unique_seqs.fa


### Step 10. ZOTU picking with UNOISE algorithm

Feed the cleaned and filtered unique sequences into the the UNOISE algorithm, which will determine the ZOTUs present.  ZOTUs are like OTUs, but more unique to the organism. It is expected that one species may have more than one ZOTU, and with 97% OTUs it is expected than an OTU may have more than one species. With the minsize parameter, only do this for sequences more than 5 bases long.

In [6]:
! usearch -unoise3 unique_seqs.fa -zotus zotus.fa -tabbedout unoise3.txt -minsize 5

usearch v11.0.667_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: cogerk@gmail.com

00:02 296Mb   100.0% Reading unique_seqs.fa
00:02 300Mb     0.0% 0 amplicons, 0 bad (size >= 113386)

00:05 349Mb   100.0% 3891 amplicons, 583629 bad (size >= 5)
00:42 350Mb   100.0% 1187 good, 2704 chimeras
00:43 350Mb   100.0% Writing zotus


### Step 10b. Relabel the header from ZOTU> OTU


In [7]:
! cat zotus.fa | sed 's/>Zo/>O/g' > otus.fa

### Step 11. Assign merged demultiplexed seqs to ZOTUs

Now that we know all the ZOTUs present, we can bin all sequences (not just unique sequences) into their respective ZOTUs. __This step takes a long time.__ 3 hours for ~20 samples.

In [32]:
awk '/target_string/ {seen = 1} seen       dd     {print}'

SyntaxError: invalid syntax (<ipython-input-32-1c7d68480abe>, line 1)

In [9]:
! usearch -otutab All_samples.fastq -zotus otus.fa -otutabout zotutab.txt -biomout zotutab.json -mapout samples.map.txt -notmatched unmapped.fasta -dbmatched zotus_with_sizes.fasta -sizeout


usearch v11.0.667_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: cogerk@gmail.com

00:00 2.6Mb   100.0% Reading otus.fa
00:00 2.6Mb   100.0% Masking (fastnucleo)
00:00 3.4Mb   100.0% Word stats
00:00 3.4Mb   100.0% Alloc rows
00:00 5.1Mb   100.0% Build index
02:16:24 37Mb    100.0% Searching, 93.1% matched
2377459 / 2552601 mapped to OTUs (93.1%)        
02:16:24 37Mb   Writing zotutab.txt
02:16:24 37Mb   Writing zotutab.txt ...done.
02:16:24 37Mb   Writing zotutab.json
02:16:24 37Mb   Writing zotutab.json ...done.


In [4]:
! head zotutab.txt

#OTU ID	AMX.MiniCol1.D0	AMX.MiniCol2.D0.AC	AMX.MiniCol3.D0.AC	AMX.MiniCol1.D31	AMX.MiniCol3.D31.AC	AMX.MiniCol2.D31.AC	KC.AC.MiniColumn.Upper	KC.AMX.Grey	KC.AMX.D46	KC.AMX.Red	KC.AC.Batch	KC.AC.MiniColumn.Mixed	KC13.C1.D121.R1	KC12.C3.D31.R2	KC6.C3.D118.R2	KC3.C2.D118.R1	KC4.C2.D118.R2	KC7.C1.D0.R2	KC10.C1.D31.R2	KC11.C2.D31.R2	KC1.C1.D118.R2b	KC5.C3.D118.R1	KC8.C2.D0.R2	KC1.C1.D118.R2	KC9.C3.D0.R2
Otu2	3954	3234	6117	2706	4806	5677	2655	4655	6926	5517	1726	2736	6074	8458	7783	7057	5614	9420	10954	7060	6443	5219	15287	3793	13436
Otu7	2485	676	1029	308	448	668	568	592	1520	2077	1611	496	7838	4083	1172	2680	1347	2027	8606	10897	2609	963	4952	1090	4806
Otu12	1648	504	731	301	483	661	822	252	364	394	958	637	11987	2689	1892	2585	1331	1616	5961	3425	2086	1856	2473	1021	2347
Otu78	67	137	83	252	404	53	136	16	4	9	11	86	346	338	101	118	331	240	106	195	346	133	128	84	85
Otu8	1006	1813	749	1808	1125	668	3803	527	2424	209	2735	2622	7467	2065	2485	1715	2440	1110	506	2397	5263	4335	584	2588	509
Otu1

In [8]:
! cp zotutab.txt zotutab.csv
pd.read_csv('zotutab.txt', sep='\t')

,#OTU ID,AMX.MiniCol1.D0,AMX.MiniCol2.D0.AC,AMX.MiniCol3.D0.AC,AMX.MiniCol1.D31,AMX.MiniCol3.D31.AC,AMX.MiniCol2.D31.AC,KC.AC.MiniColumn.Upper,KC.AMX.Grey,KC.AMX.D46,...,KC3.C2.D118.R1,KC4.C2.D118.R2,KC7.C1.D0.R2,KC10.C1.D31.R2,KC11.C2.D31.R2,KC1.C1.D118.R2b,KC5.C3.D118.R1,KC8.C2.D0.R2,KC1.C1.D118.R2,KC9.C3.D0.R2
0,Otu2,3954,3234,6117,2706,4806,5677,2655,4655,6926,...,7057,5614,9420,10954,7060,6443,5219,15287,3793,13436
1,Otu7,2485,676,1029,308,448,668,568,592,1520,...,2680,1347,2027,8606,10897,2609,963,4952,1090,4806
2,Otu12,1648,504,731,301,483,661,822,252,364,...,2585,1331,1616,5961,3425,2086,1856,2473,1021,2347
3,Otu78,67,137,83,252,404,53,136,16,4,...,118,331,240,106,195,346,133,128,84,85
4,Otu8,1006,1813,749,1808,1125,668,3803,527,2424,...,1715,2440,1110,506,2397,5263,4335,584,2588,509
5,Otu11,697,621,795,813,1020,855,593,1638,109,...,1451,6820,1539,1342,1858,5136,2333,2261,4171,1977
6,Otu17,2158,1043,1713,890,754,1508,283,195,121,...,763,517,1385,2265,2404,1551,949,2754,612,1266
7,Otu62,130,178,143,146,216,128,177,67,79,...,341,272,329,168,686,402,299,182,235,129
8,Otu40,148,302,71,387,417,78,30,24,33,...,568,684,555,200,1191,1914,870,77,774,65
9,Otu367,13,13,2,6,8,6,2,3,5,...,26,13,8,4,12,11,24,12,4,2


### Step 12. Normalize samples 

| Sample Name            | No. Sequences|
|------------------------|--------------|
| KC.AC.MiniColumn.Upper | 10473        |
| KC.AC.MiniColumn.Mixed | 8505         |
| KC.AC.Batch            | 9388         |
| KC.AMX.Red             | 8453         |
| KC.AMX.Grey            | 10533        |
| KC.AMX.D46             | 13626        |
| Tacoma_suspended       | 40109        |

Maximum number of sequences is 8,453, we will normalize to 8,400 sequences in all samples

### Step 13. Assign taxonomies to OTUs

Using silva database predict taxonomy (already cleaned up by Sam), using a cutoff of 80% confidence. We will make the database using from the fasta file.


In [120]:
# Revise uncultured genuses to be uncultured <Family>

# Open the file with read only permit
#f = open('silva.seed_v132.usearch.fasta', "r")
# use readlines to read all lines in the file
# The variable "lines" is a list containing all lines in the file
#lines = f.readlines()
# close the file after reading the lines.
#f.close()
#myfile = open('rev.silva.seed_v132.usearch.fasta', 'w')
#for i, line in enumerate(lines):
#    if line[0] == '>':
#        if ':uncultured' in line:
#            uncul = line.split(',')[-2].split(':')[1]
#            line = line.replace(':uncultured', ':uncultured ' + uncul)
#    myfile.writelines(line) 
#myfile.close()


In [6]:
! usearch10 -makeudb_sintax rdp_16s_v16.fa -output rdp_16s_v16.udb



usearch v10.0.240_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-17 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: cogerk@gmail.com

00:01 45Mb    100.0% Reading rdp_16s_v16.fa
00:01 45Mb    100.0% Converting to upper case
00:01 46Mb    100.0% Word stats
00:01 46Mb    100.0% Alloc rows
00:02 120Mb   100.0% Build index
00:02 121Mb   100.0% Initialize taxonomy data
00:02 121Mb   100.0% Building name table
00:02 121Mb  3172 names, tax levels min 3, avg 5.9, max 6
00:03 122Mb  Buffers (13212 seqs)
00:03 140Mb   100.0% Seqs


In [7]:
! usearch10 -sintax otus.fa -db rdp_16s_v16.udb -tabbedout otus.sintax -strand both -sintax_cutoff 0.8



usearch v10.0.240_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-17 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: cogerk@gmail.com

00:00 83Mb    100.0% Rows
00:00 83Mb   Read taxonomy info...done.
00:00 84Mb   Reading pointers...done.
00:00 86Mb   Reading db seqs...done.
00:08 106Mb   100.0% Processing
